# Playing with (DNB) data

Today, we will be working with a public data set and a public data set that is published by DNB. Finally, for the people that finish early, there is some room to experiment with a simple neural network. We will use this Jupyter notebook to work with both code and rich text elements, such as figures, links, equations, ... Because of the mix of code and text elements, these documents are the ideal place to bring together an analysis description and its results as well as they can be executed perform the data analysis in real time.

You will have to fill in the blanks at some points, run the code at others. You are free to change code or experiment. Make sure to ask us for help if you cannot come up with the solution or get stuck somewhere!

Some things you should know about the Jupyter notebook:

Each block that you navigate using the up and down keys is called a cell
* Hit [Shift+Enter] to execute a cell
* Hit [Enter] to edit a cell in the Edit mode. Once you are done, hit [Escape] to come back into command mode
* For a list of all shortcuts, when in command mode, hit [Ctrl+H]

Try running the cells below using the methods suggested above. You can edit the code if you like.

In [ ]:
print('Hello World')

In [ ]:
3%120

---

## Let's get started

Great, now we're ready to start. First, we need some libraries that we use throughout this notebook. These libraries range from data analysis libraries such as Pandas to plotting libraries such as matplotlib. 

In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('fast')
plt.rcParams['figure.figsize'] = [12, 12]

In [ ]:
import pandas as pd
import numpy as np
from load_solvency import read_sheet
from datetime import datetime
from sklearn.manifold import TSNE

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils

from fbprophet import Prophet

## Read in the data

Before moving on to analysis, we need to load the data first from the internet and shape it into the right format. Let's read in data on the Euribor, the Euro Interbank Offer Rate*. We will first define the url that points to the data that we need and then use it to read the data into a Pandas dataframe.

In [2]:
url = 'https://www.emmi-benchmarks.eu/assets/modules/rateisblue/file_processing/publication/processed/hist_EURIBOR_2018.csv' 

In [3]:
url

'https://www.emmi-benchmarks.eu/assets/modules/rateisblue/file_processing/publication/processed/hist_EURIBOR_2018.csv'

In [ ]:
euribor = pd.read_csv(url, index_col=[0])

Let's take a look at the first 10 rows.

In [ ]:
euribor.head(10)

Personally, I think it makes more sense to transpose the dataframe. 

In [ ]:
euribor = euribor.T

In [ ]:
euribor.head(3)

There is a convenient function called ``describe()`` that automatically calculates some basic statistics on the ``euribor`` data set.

Usually, date columns are not parsed automatically, so we need to tell pandas to store the index column as date format. 

In [ ]:
euribor.index = pd.to_datetime(euribor.index, dayfirst=True)

In [ ]:
euribor.describe()

## Visualize the data

Let's take a visual look at the data. You are free to experiment with plotting parameters.

In [ ]:
euribor.plot(ylim=[-0.5,0], linewidth=2, figsize=(10,10))

## Predict the future benchmark rate

Maybe we would want to forecast how one of the benchmark rates will move in the future. One well-known way of doing this, would be using an [ARIMA model](https://en.wikipedia.org/wiki/Autoregressive_integrated_moving_average) to do 'time series forecasting'. Another way that we will use here is a forecasting method that was developed by Facebook called [Prophet](https://facebook.github.io/prophet/), which they use in production. Be sure to read their paper if you are interested to find out the technical details. In this notebook, you will see how easy it is to use the model.

Say we would like to know what the 1 year euribor will do in the future based on the data we have just loaded in. We first need to get the data in the right format for Prophet to be able to use the data: it expects a Dataframe with one column `ds` and one column `y`, for the dates and rates respectively. You can also change the tenor to one of the other available tenors, e.g. `6m`, `9m`, ...

In [ ]:
df = euribor['12m'].reset_index() # to convert the index column to a 'regular' column
df.columns = ['ds', 'y']

In [ ]:
df.head(5)

We can now initialize the model and fit it to our prepared data set `df`.

In [ ]:
m = Prophet()
m.fit(df)

The model is now trained on our data. Let's create a dataframe with 180 days in the future to predict the future rate.

In [ ]:
future = m.make_future_dataframe(periods=180)
future.tail()

Let's predict.

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat']].tail()

And plot our prediction.

In [ ]:
m.plot(forecast)
plt.show()

---

## Your turn: Solvency II data

Up and till now, you have been able to sit back and [Shift-Enter]. Now - if you have time left... - try and reuse parts of what you learned in this section to complete the exercises below. This time you'll be using a (limited) public data set published on DNB's website.

Let's start by reading in a new data set. The name of the data file is `Individuele gegevens verzekeraars per jaar.xlsx`. It contains information from https://statistiek.dnb.nl/downloads/index.aspx#/details/individuele-gegevens-verzekeraars-jaar/dataset/d66feb58-e89b-4c73-b0c7-cab81188ca77 and contains public information about insurance companies. More specifically, we use public Solvency II data from individual Dutch insurance undertakings.

We will walk through similar steps as before, but now, you will need to actually write some code and fill code cells to analyze the data. If you get stuck, scroll up to the relevant code you ran earlier.

Let's start by reading in the data using the name given above. 

**Exercise**: Edit line 1 such that you load in the correct file. Keep line 2 as is.

In [ ]:
xls = pd.ExcelFile('Individuele gegevens verzekeraars per jaar.xlsx')
df = read_sheet(14, xls)

**Exercise:** Take a peak at the data in `df` using the `head` function.

In [ ]:
df.head(3)

The data frame `df` now contains the data of the balance sheets of all Dutch insurance undertakings. Because Solvency II came into force in 2016 we have two years of data (per 2016-12-31 and per 2017-12-31).

Suppose we want to do some calculations with the balance sheets per 2017-12-31. 

Then we have to select this date from the data frame. For this we use the command `xs` of the `df` object. The parameters are `axis = 0` (we select from the rows), `level = 1` (level 0 is the name of the insurance undertaking and level 1 is the date of the report).

In [ ]:
df = df.xs(datetime(2017,12,31), axis = 0, level = 1)

Let's see the number of insurance companies of which we have the balance sheet and how many columns of information we have:

In [ ]:
nr_of_i = len(df)
nr_of_cols = len(df.columns)
print(f'The number of insurance companies in our data set is {nr_of_i} and the number of columns is {nr_of_cols}')

Time to find out some general things. 

**Exercise:** Calculate the sum of all total assets by selecting the `total assets` column and applying the `sum()` function and calculate the maximum `total assets`.

In [ ]:
df['total assets'].sum()

On to looking at insurance premiums. These are in sheet 16 of the Excel sheet we are taking a look at, so we are going to load that.

In [ ]:
df_premiums = read_sheet(16, xls)
df_premiums = df_premiums.xs(datetime(2017,12,31), level = 1)

We will now use a specific algorithm to visualize the high-dimensional data: [t-Distributed Stochastic Neighbor Embedding](https://lvdmaaten.github.io/tsne/). 

In [ ]:
X = df_premiums.values
Y = TSNE(n_components = 2, perplexity = 9, 
         verbose = 1, random_state = 1).fit_transform(X)

Finally, we can produce a scatter plot based on the results the t-SNE algorithm gives us. Uncomment the two commented lines to add the names of the insurance companies to the plot. You can play with t-SNE's perplexity value to see how the clusters change.

In [ ]:
plt.scatter(x = Y[:, 0], 
              y = Y[:, 1], 
              s = 20)

# for i in range(len(Y[:,0])):
#     plt.annotate(df_premiums.index[i][0:20], (Y[i,0], Y[i,1]))

---

# Next level

## Building a simple neural-network with Keras¶

If you have any time left, walk through the following cells of code which builds a simple neural network that is learned to recognize digits. It is trained on the well known [MNIST](http://yann.lecun.com/exdb/mnist/) data set. 

This part is from https://github.com/wxs/keras-mnist-tutorial/blob/master/MNIST%20in%20Keras.ipynb, a short tutorial given at the University of Toronto, which itself is based on a [Keras example](https://github.com/keras-team/keras/blob/master/examples/mnist_mlp.py).

### Load training data

Let's look at some examples of the training data

In [ ]:
nb_classes = 10 # 10 digits

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)

Let's look at some examples of the training data by plotting them.

In [ ]:
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(X_train[i], cmap='gray', interpolation='none')
    plt.title("Class {}".format(y_train[i]))

### Format the data for training

Our neural-network is going to take a single vector for each training example, so we need to reshape the input so that each 28x28 image becomes a single 784 dimensional vector. We'll also scale the inputs to be in the range [0-1] rather than [0-255]. This is done because neural networks are known to work better on normalized data.

In [ ]:
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)


Modify the target matrices to be in the one-hot format, i.e.


    
    0 -> [1, 0, 0, 0, 0, 0, 0, 0, 0],
    1 -> [0, 1, 0, 0, 0, 0, 0, 0, 0],
    2 -> [0, 0, 1, 0, 0, 0, 0, 0, 0],
    etc.
       

In [ ]:
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

Build the neural-network. Here we'll do a simple 3 layer fully connected network.

![Neural Net](figure.png)

In [ ]:
model = Sequential()
model.add(Dense(512, input_shape=(784,)))
model.add(Activation('relu')) # An "activation" is just a non-linear function applied to the output
                              # of the layer above. Here, with a "rectified linear unit",
                              # we clamp all values below 0 to 0.
                           
model.add(Dropout(0.2))   # Dropout helps protect the model from memorizing or "overfitting" the training data
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax')) # This special "softmax" activation among other things,
                                 # ensures the output is a valid probaility distribution, that is
                                 # that its values are all non-negative and sum to 1.

### Compile the model

Keras is built on top of Theano (and now TensorFlow as well), both packages that allow you to define a computation graph in Python, which they then compile and run efficiently on the CPU or GPU without the overhead of the Python interpreter.

When compiling a model, Keras asks you to specify your loss function and your optimizer. The loss function we'll use here is called categorical crossentropy, and is a loss function well-suited to comparing two probability distributions.

Here our predictions are probability distributions across the ten different digits (e.g. "we're 80% confident this image is a 3, 10% sure it's an 8, 5% it's a 2, etc."), and the target is a probability distribution with 100% for the correct category, and 0 for everything else. The cross-entropy is a measure of how different your predicted distribution is from the target distribution. More detail at Wikipedia

The optimizer helps determine how quickly the model learns, how resistent it is to getting "stuck" or "blowing up". We won't discuss this in too much detail, but "adam" is often a good choice (developed here at U of T).

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

### Train the model

This is the fun part: you can feed the training data loaded in earlier into this model and it will learn to classify digits.

In [ ]:
model.fit(X_train, Y_train,
          batch_size=128, epochs=4, verbose=1,
          validation_data=(X_test, Y_test))

### Finally, evaluate its performance.

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print(score)

### Inspecting the output

It's always a good idea to inspect the output and make sure everything looks sane. Here we'll look at some examples it gets right, and some examples it gets wrong.

In [ ]:
# The predict_classes function outputs the highest probability class
# according to the trained classifier for each input example.
predicted_classes = model.predict_classes(X_test)

# Check which items we got right / wrong
correct_indices = np.nonzero(predicted_classes == y_test)[0]
incorrect_indices = np.nonzero(predicted_classes != y_test)[0]

In [ ]:
plt.figure()
for i, correct in enumerate(correct_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(X_test[correct].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[correct], y_test[correct]))
    
plt.figure()
for i, incorrect in enumerate(incorrect_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(X_test[incorrect].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[incorrect], y_test[incorrect]))

# Glossary & References

* Euribor, or the Euro Interbank Offer Rate, is a reference rate that is constructed from the average interest rate at which eurozone banks offer unsecured short-term lending on the inter-bank market (https://www.investopedia.com/terms/e/euribor.asp).
* [Python Exercises](https://github.com/iitmcvg/Python-Exercises/)
* [Prophet: forecasting at scale](https://research.fb.com/prophet-forecasting-at-scale/)
* Keras with MNIST: https://github.com/wxs/keras-mnist-tutorial/blob/master/MNIST%20in%20Keras.ipynb

